In [1]:
# Standard libraries
import datetime
import math
import pathlib
import re
import requests
import sqlite3
import sys

# Installed libraries
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import spiceypy
import tqdm

# Append to root directory of this repository
sys.path.append("../")

# Auxiliary module that contains the apparent magnitude
from auxiliary import photometry

In [2]:
# Let's set a dark background
plt.style.use('dark_background')

# Set a default font size for better readability
plt.rcParams.update({'font.size': 14})

In [3]:
# Load SPICE kernels
spiceypy.furnsh("../kernels/spk/de432s.bsp")
spiceypy.furnsh("../kernels/lsk/naif0012.tls")
spiceypy.furnsh("../kernels/pck/gm_de431.tpc")

# Get the G*M value of the Sun
_, gm_sun_pre = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)
gm_sun = gm_sun_pre[0]

In [4]:
# Load the Granvik model
data_dir = pathlib.Path("results_data/Granvik")
data_dir.mkdir(parents=True, exist_ok=True)
dataframe_filepath = data_dir / "enriched_granvik_model.parquet"

granvik_model_df = pd.read_parquet(dataframe_filepath)

In [5]:
granvik_model_df

,SemiMajorAxis_AU,Ecc_,Incl_deg,LongAscNode_deg,ArgP_deg,H,Perihelion_AU,Incl_rad,LongAscNode_rad,ArgP_rad,Perihelion_km,SemiMajorAxis_km,epoch_et,MeanAnom_deg,MeanAnom_rad
0,2.574981,0.783617,33.520763,278.480591,75.952057,21.064367,0.557182,0.585048,4.860403,1.325613,8.335328e+07,3.852117e+08,64.183927,0,0.000000
1,2.574981,0.783617,33.520763,278.480591,75.952057,21.064367,0.557182,0.585048,4.860403,1.325613,8.335328e+07,3.852117e+08,64.183927,60,1.047198
2,2.574981,0.783617,33.520763,278.480591,75.952057,21.064367,0.557182,0.585048,4.860403,1.325613,8.335328e+07,3.852117e+08,64.183927,120,2.094395
3,2.574981,0.783617,33.520763,278.480591,75.952057,21.064367,0.557182,0.585048,4.860403,1.325613,8.335328e+07,3.852117e+08,64.183927,180,3.141593
4,2.574981,0.783617,33.520763,278.480591,75.952057,21.064367,0.557182,0.585048,4.860403,1.325613,8.335328e+07,3.852117e+08,64.183927,240,4.188790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4811995,2.591117,0.826896,22.001493,334.001678,286.224030,24.542505,0.448532,0.383999,5.829429,4.995552,6.709938e+07,3.876256e+08,64.183927,60,1.047198
4811996,2.591117,0.826896,22.001493,334.001678,286.224030,24.542505,0.448532,0.383999,5.829429,4.995552,6.709938e+07,3.876256e+08,64.183927,120,2.094395
4811997,2.591117,0.826896,22.001493,334.001678,286.224030,24.542505,0.448532,0.383999,5.829429,4.995552,6.709938e+07,3.876256e+08,64.183927,180,3.141593
4811998,2.591117,0.826896,22.001493,334.001678,286.224030,24.542505,0.448532,0.383999,5.829429,4.995552,6.709938e+07,3.876256e+08,64.183927,240,4.188790


# TBD

In [6]:
init_et = granvik_model_df.iloc[0]["epoch_et"]

sun2earth_position_vec = spiceypy.spkgps(targ=399,
                                         et=init_et,
                                         ref="ECLIPJ2000",
                                         obs=10)[0]

In [7]:
tqdm.tqdm.pandas()

# Compute the position vector of each NEO as seen from the Sun
granvik_model_df.loc[:, "sun2neo_position_vec"] = \
    granvik_model_df.progress_apply(lambda x: spiceypy.conics(elts=[x["Perihelion_km"],
                                                 x["Ecc_"],
                                                 x["Incl_rad"],
                                                 x["LongAscNode_rad"],
                                                 x["ArgP_rad"],
                                                 x["MeanAnom_rad"],
                                                 x["epoch_et"],
                                                 gm_sun],
                                         et=init_et)[:3],
                 axis=1)

100%|██████████████████████████████| 4812000/4812000 [02:23<00:00, 33472.73it/s]


In [8]:
# To compute the apparent magnitude we need to re-compute the positional vectors and convert it to
# AU
granvik_model_df.loc[:, "neo2earth_position_vec"] = \
    granvik_model_df["sun2neo_position_vec"].progress_apply(lambda x: sun2earth_position_vec - x)
granvik_model_df.loc[:, "neo2sun_position_vec"] = \
    granvik_model_df["sun2neo_position_vec"].progress_apply(lambda x: -1.0 * x)

granvik_model_df.loc[:, "neo2earth_position_vec_AU"] = \
    granvik_model_df["neo2earth_position_vec"].progress_apply(lambda x: [spiceypy.convrt(k, "km", "AU") for k in x])
granvik_model_df.loc[:, "neo2sun_position_vec_AU"] = \
    granvik_model_df["neo2sun_position_vec"].progress_apply(lambda x: [spiceypy.convrt(k, "km", "AU") for k in x])

100%|██████████████████████████████| 4812000/4812000 [01:04<00:00, 74316.53it/s]


In [9]:
# Compute the apparent magnitude of each NEO for today!
granvik_model_df.loc[:, "app_mag"] = \
    granvik_model_df.progress_apply(lambda x: photometry.hg_app_mag(abs_mag=x["H"],
                                                 vec_obj2obs=x["neo2earth_position_vec_AU"],
                                                 vec_obj2ill=x["neo2sun_position_vec_AU"],
                                                 slope_g=0.15), axis=1)

100%|██████████████████████████████| 4812000/4812000 [02:25<00:00, 33022.46it/s]


In [10]:
# We compute now the NEO's coordiantes in a similar way using the dataframe and the apply function.
granvik_model_df.loc[:, "earth2neo_position_vec_AU"] = \
    granvik_model_df["neo2earth_position_vec_AU"].progress_apply(lambda x: -1.0 * np.array(x))

granvik_model_df.loc[:, "earth2neo_recrad"] = \
    granvik_model_df["earth2neo_position_vec_AU"].progress_apply(lambda x: spiceypy.recrad(x))

granvik_model_df.loc[:, "earth2neo_dist_AU"] = \
    granvik_model_df["earth2neo_recrad"].progress_apply(lambda x: x[0])

granvik_model_df.loc[:, "earth2neo_eclip_long"] = \
    granvik_model_df["earth2neo_recrad"].progress_apply(lambda x: x[1])

granvik_model_df.loc[:, "earth2neo_eclip_lat"] = \
    granvik_model_df["earth2neo_recrad"].progress_apply(lambda x: x[2])

100%|████████████████████████████| 4812000/4812000 [00:03<00:00, 1378644.08it/s]


In [11]:
granvik_model_df.head()

,SemiMajorAxis_AU,Ecc_,Incl_deg,LongAscNode_deg,ArgP_deg,H,Perihelion_AU,Incl_rad,LongAscNode_rad,ArgP_rad,...,neo2earth_position_vec,neo2sun_position_vec,neo2earth_position_vec_AU,neo2sun_position_vec_AU,app_mag,earth2neo_position_vec_AU,earth2neo_recrad,earth2neo_dist_AU,earth2neo_eclip_long,earth2neo_eclip_lat
0,2.574981,0.783617,33.520763,278.480591,75.952057,21.064367,0.557182,0.585048,4.860403,1.325613,...,"[-96159838.5522258, 154766801.0385216, -446549...","[-69658892.50602701, 10069855.30884874, -44654...","[-0.6427888188364811, 1.0345521657736734, -0.2...","[-0.46564093606592355, 0.06731282515947325, -0...",22.119561,"[0.6427888188364811, -1.0345521657736734, 0.29...","(1.2540245866828015, 5.268338230531873, 0.2403...",1.254025,5.268338,0.240340
1,2.574981,0.783617,33.520763,278.480591,75.952057,21.064367,0.557182,0.585048,4.860403,1.325613,...,"[288365432.4872961, -132825704.89924341, 17917...","[314866378.5334949, -277522650.62891626, 17917...","[1.9276038576240895, -0.8878849969879803, 1.19...","[2.104751740394647, -1.8551243376021804, 1.197...",26.295329,"[-1.9276038576240895, 0.8878849969879803, -1.1...","(2.4369147775528055, 2.709945690847877, -0.513...",2.436915,2.709946,-0.513805
2,2.574981,0.783617,33.520763,278.480591,75.952057,21.064367,0.557182,0.585048,4.860403,1.325613,...,"[483189339.56666315, -64347883.79746801, 31350...","[509690285.61286193, -209044829.52714086, 3135...","[3.2299212387482283, -0.43013903562595157, 2.0...","[3.407069121518786, -1.3973783762401517, 2.095...",27.900894,"[-3.2299212387482283, 0.43013903562595157, -2....","(3.8741807177808067, 3.009198433910259, -0.571...",3.874181,3.009198,-0.571548
3,2.574981,0.783617,33.520763,278.480591,75.952057,21.064367,0.557182,0.585048,4.860403,1.325613,...,"[547688149.7746499, 61692451.87651107, 3680788...","[574189095.8208487, -83004493.85316178, 368079...","[3.6610691551149253, 0.41238856959282105, 2.46...","[3.838217037885484, -0.5548507710213791, 2.460...",28.344639,"[-3.6610691551149253, -0.41238856959282105, -2...","(4.430274455989257, 3.253761412328597, -0.5888...",4.430274,3.253761,-0.588812
4,2.574981,0.783617,33.520763,278.480591,75.952057,21.064367,0.557182,0.585048,4.860403,1.325613,...,"[498681916.0357363, 204141020.98990104, 349887...","[525182862.0819351, 59444075.26022819, 3498878...","[3.333482715963904, 1.3645984408231357, 2.3388...","[3.510630598734462, 0.39735910020893567, 2.338...",28.134247,"[-3.333482715963904, -1.3645984408231357, -2.3...","(4.294701880301309, 3.530142879771741, -0.5759...",4.294702,3.530143,-0.575900


In [12]:
granvik_model_df.drop(columns=["neo2earth_position_vec",
                               "neo2sun_position_vec",
                               "neo2earth_position_vec_AU",
                               "neo2sun_position_vec_AU",
                               "earth2neo_position_vec_AU",
                               "earth2neo_recrad"],
                      inplace=True)

In [13]:
neo_positions = granvik_model_df[["earth2neo_eclip_lat",
                                  "earth2neo_eclip_long"]].values
neo_positions

array([[ 0.24034044,  5.26833823],
       [-0.51380492,  2.70994569],
       [-0.57154774,  3.00919843],
       ...,
       [ 0.45263439,  1.32980167],
       [ 0.43434075,  1.57543327],
       [ 0.39869459,  1.9652046 ]])

In [16]:
dataframe_filepath = data_dir / "enriched_granvik_model_w_appmag.parquet"

granvik_model_df.to_parquet(dataframe_filepath)